# Sensitivity Analysis
- No Shock on RL model (RW)
- Sensomotor cortex
- Framewise displacement
- Compare different diagnositc groups
- Play with different delays

## No shock RL model
Here we will use only the "winning" model (RW)

In [2]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pytensor
import pytensor.tensor as pt
import scipy
import os
# import stan

import pymc as pm
import arviz as az
# import learning package
#import DMpy

In [95]:
# read scr file
#scr = pd.read_csv('/media/Data/Lab_Projects/PTSD_Reversal/Behavioral/SCR3.csv')
scr = pd.read_csv('behavior/SCR3.csv')
scr.head()


,Event.Nr,CDA.nSCR,CDA.Latency,CDA.AmpSum,CDA.SCR,CDA.ISCR,CDA.PhasicMax,CDA.Tonic,TTP.nSCR,TTP.Latency,TTP.AmpSum,Global.Mean,Global.MaxDeflection,Event.NID,Event.Name,Condition,group,sub
0,1,5,0.8435,0.2852,0.0003,0.1339,8.1296,2.3324,1,3.9335,0.5884,2.4822,0.5884,5,5,CSplusUS1,HC,189
1,2,4,0.7335,0.1033,0.0012,0.4737,0.3046,4.0029,0,NaN,0.0000,4.3933,0.0000,5,5,CSminus1,HC,189
2,3,3,2.9835,0.0783,0.0008,0.3237,0.1352,3.9579,1,2.8335,0.0260,3.9900,0.0154,5,5,CSplus1,HC,189
3,4,1,3.4935,0.1772,0.0002,0.0993,7.0748,3.8756,1,3.5335,0.5186,3.9212,0.5186,5,5,CSplusUS1,HC,189
4,5,0,NaN,0.0000,0.0004,0.1532,0.1604,4.2513,0,NaN,0.0000,4.3461,0.0000,5,5,CSminus1,HC,189


In [96]:
scr = scr[['sub','Condition','Event.Nr','CDA.AmpSum']]
scr['sub'] = scr['sub'].astype('string')
for i in scr.iterrows():
    if len(i[1]['sub'])<=2:
        #print(i[1]['sub'])
        sub = 'sub-0' + str(i[1]['sub'])
    else:
        sub = 'sub-' + str(i[1]['sub'])
    #print(sub)
    scr.at[i[0], 'sub'] = sub
    

In [97]:
# grab subjects with 69 trials
scr_clean = scr.copy() # make a copy of original
for sub in scr['sub'].unique():
    df = scr[scr['sub']==sub]
    #print (len(df))
    if len(df)<69:
        scr_clean = scr_clean[scr_clean['sub']!=sub]
        
len(scr_clean['sub'].unique()) # total of 86 valid subjects


86

In [98]:
# grab just two subjects for now
scrTwo = scr_clean#[(scr['sub']==152) |(scr['sub']==189) | (scr['sub']==86) | (scr['sub']==48)]
scrTwo['Event.Nr'].values

array([ 1,  2,  3, ..., 67, 68, 69], shape=(5934,))

In [99]:
# organize data accordingly
# first, grab just the relevant variables (subject, trial, stimuli, shock)
# we need to generate a new variable shock (1=yes, 0=no)
# we also need a new variable stim (1=CS+, 2=CS-)
scrVec = scrTwo['CDA.AmpSum'].values
print(scrVec.shape)
shockVec = np.zeros(len(scrVec), dtype=np.int32) # vector to capture shock (1=yes, 0=no)
stimVec = np.zeros(len(scrVec), dtype=np.int32) # vector to capture stimulus (1=CS+, 2= CS-)

for i, cond in enumerate(scrTwo['Condition'].values):
    if cond=='CSplusUS1':
        shockVec[i]= 1
        stimVec[i] = 1
    elif cond=='CSminusUS2':
            # after reversal so minus becomes plus
        shockVec[i]= 1
        stimVec[i] = 0
    elif cond=='CSplus2':
            # after reversal so plus becomes minus
            shockVec[i]=0
            stimVec[i] = 1
    elif cond=='CSplus1':
            stimVec[i] = 1
            shockVec[i]= 0
    elif cond=='CSminus1':
            stimVec[i] = 0
            shockVec[i] = 0
    elif cond=='CSminus2':
        stimVec[i] = 0
        shockVec[i] = 0
    else:
        print(f' Condition is {cond}')
        stimVec[i] = 9
        shockVec[i] = 9
print(shockVec.shape)
print(stimVec.shape)

(5934,)
(5934,)
(5934,)


In [100]:
# get index of shocks
np.argwhere(shockVec==1)[:,0]

array([   0,    3,    8, ..., 5914, 5920, 5928], shape=(1118,))

In [101]:
sum(shockVec) / sum(stimVec) 

np.float64(0.38235294117647056)

In [102]:
n_trials, n_subj = 69,len(scrTwo['sub'].unique())
trials, subj = np.meshgrid(range(n_trials), range(n_subj))

In [103]:
n_trials, n_subj = 69,len(scrTwo['sub'].unique())
trials, subj = np.meshgrid(range(n_trials), range(n_subj))

trials = pt.as_tensor_variable(trials.T)
subj = pt.as_tensor_variable(subj.T)
stim = np.reshape(stimVec, (n_subj,n_trials)).T # transform to matrix trials x subject
shock = np.reshape(shockVec, (n_subj,n_trials)).T 
scrMat = np.reshape(scrVec, (n_subj, n_trials)).T

In [104]:
stim = pt.as_tensor_variable(stim)
shock = pt.as_tensor_variable(shock)


# Build PyMC code

In [105]:
#scrs = pt.zeros(30) # set empty scr tensor (vector)
   
# generate functions to run
def update_Q(stim, shock,
             Qs,vec,
             alpha, n_subj):
    """
    This function updates the Q table according to the RL update rule.
    It will be called by pytensor.scan to do so recursevely, given the observed data and the alpha parameter
    This could have been replaced be the following lamba expression in the pytensor.scan fn argument:
        fn=lamba action, reward, Qs, alpha: pt.set_subtensor(Qs[action], Qs[action] + alpha * (reward - Qs[action]))
    """
      
    PE = shock - Qs[pt.arange(n_subj), stim]
    Qs = pt.set_subtensor(Qs[pt.arange(n_subj),stim], Qs[pt.arange(n_subj),stim] + alpha * PE)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in pytensor)
    vec = pt.set_subtensor(vec[pt.arange(n_subj),0], (pt.switch(pt.eq(stim,1), 
                                                                Qs[pt.arange(n_subj),1], Qs[pt.arange(n_subj),0])))
    
    return Qs, vec, PE

In [106]:
# removing shocks
# remove shocks from SCR vector (feed the model)
scrClean = scrVec[shockVec==0]

In [107]:
with pm.Model() as m5:
    
    # α
    phi = pm.Uniform("phi", lower=0.0, upper=1.0)
    kappa_log = pm.Exponential("kappa_log", lam=1.5)
    kappa = pm.Deterministic("kappa", pt.exp(kappa_log))
    alpha = pm.Beta("alpha", alpha=phi * kappa, beta=(1.0 - phi) * kappa, shape=n_subj)
    
    # β
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
       
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 0.5 * pt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 0.5 * pt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec, pe], updates = pytensor.scan(
        fn=update_Q,
        sequences=[stim, shock],
        outputs_info=[Qs, vec, None],
        non_sequences=[alpha, n_subj])
   
     
    vec_ = vec[trials, subj,0] * beta[subj]
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    # add PE
    pe = pm.Deterministic('pe', pe)

    # transform to vector
    v = pt.reshape(vec_.T, n_subj*n_trials, ndim=1)
    # clean shocks
    vec_clean = v[shockVec==0]
       
    # likelihood function
    scrs = pm.Normal('scrs', mu = vec_clean, sigma = eps, observed=scrClean) 
   
    trH_noShock = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                   nuts_sampler="blackjax"
                  )

Running window adaptation


In [108]:
# stacking the PE
mean_pe = trH_noShock.posterior.stack(draws=('chain','draw')).pe
mean_pe = np.mean(mean_pe, axis=2)

scr_clean['sub'].unique()
mean_pe_vec = np.reshape(mean_pe.T, (69*86,1)) # reshape PE
scr_clean['pe'] = mean_pe_vec.values
scr_clean['scr'] = scr_clean['CDA.AmpSum']
# save file
scr_clean.to_csv('scr_clean_noShocks.csv', index=False)
# loading coupling data
amg_hipp = pd.read_csv('amg_hipp_fc_WholeROIs.csv')
# merging with scr and PE data
dfAll = pd.merge(scr_clean, amg_hipp, right_on=['subject','trialNo'], left_on=['sub','Event.Nr'])
# saving combined data for future analysis
dfAll.to_csv('scr_amg_hipp_all_noShock.csv', index=False)

### Now we run the Statistical Analysis for Amygdala-Hippocampus

In [109]:
df = pd.read_csv('scr_amg_hipp_all_noShock.csv')
# Run Pymc model
# Encode 'sub' as integer indices
df['sub_idx'] = pd.Categorical(df['sub']).codes
n_subs = df['sub_idx'].nunique()

# Extract variables
pe = df['pe'].values
coupling = df['coupling'].values
amg = df['amg'].values
trialNo = df['trialNo'].values
sub_idx = df['sub_idx'].values

with pm.Model() as model:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]
/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Output()

/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9 seconds.


In [110]:
az.summary(trace, hdi_prob = 0.89)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_coupling,0.096,0.019,0.067,0.128,0.000,0.000,5504.0,2805.0,1.0
beta_amg,0.010,0.023,-0.026,0.047,0.000,0.000,7345.0,2536.0,1.0
beta_trialNo,0.001,0.000,0.001,0.002,0.000,0.000,4587.0,2725.0,1.0
mu_a,-0.216,0.016,-0.240,-0.189,0.000,0.000,3822.0,2783.0,1.0
z_a[0],0.090,0.949,-1.436,1.568,0.011,0.017,6850.0,2604.0,1.0
...,...,...,...,...,...,...,...,...,...
a[60],-0.222,0.021,-0.255,-0.190,0.000,0.000,3581.0,2711.0,1.0
a[61],-0.213,0.020,-0.245,-0.181,0.000,0.000,3815.0,2558.0,1.0
a[62],-0.214,0.020,-0.244,-0.182,0.000,0.000,4092.0,2684.0,1.0
a[63],-0.216,0.021,-0.251,-0.185,0.000,0.000,3841.0,2187.0,1.0


In [111]:
# same for vmpfc
coupling = df['amg_vmpfc']

with pm.Model() as model_v:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace_vmpfc = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]
/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Output()

/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.


In [112]:
az.summary(trace_vmpfc, hdi_prob = .89)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_coupling,-0.040,0.016,-0.064,-0.014,0.00,0.000,7578.0,2631.0,1.0
beta_amg,0.006,0.022,-0.032,0.040,0.00,0.000,8027.0,3036.0,1.0
beta_trialNo,0.002,0.000,0.001,0.002,0.00,0.000,5414.0,3188.0,1.0
mu_a,-0.152,0.013,-0.173,-0.132,0.00,0.000,5141.0,2876.0,1.0
z_a[0],0.014,0.957,-1.490,1.620,0.01,0.016,9180.0,3131.0,1.0
...,...,...,...,...,...,...,...,...,...
a[60],-0.156,0.018,-0.183,-0.129,0.00,0.000,4491.0,3230.0,1.0
a[61],-0.150,0.017,-0.177,-0.125,0.00,0.000,4823.0,3031.0,1.0
a[62],-0.152,0.016,-0.177,-0.126,0.00,0.000,5221.0,3272.0,1.0
a[63],-0.151,0.016,-0.177,-0.126,0.00,0.000,5057.0,3373.0,1.0


## Summary:
Removing shocks from the RL model doesn't change the results.


## Sensorimotor Cortex

In [37]:
# grab sensorimotor to the data
import glob
import os
sm_files = glob.glob('timeseries/sensoryMotot_sub-*.csv')
tsFiles =  glob.glob('timeseries/small_amg_hipp_vmpfc_sub-*.csv')

'sub-001'

In [52]:
ts_total57 = [] # the 570 timeseries
ts_total55 = [] # the 555
ts_total51 = [] # the 514
sub51 = []
sub55 = []
sub57 = []
for f in tsFiles:
    ts = pd.read_csv(f)
    sub = os.path.basename(f).split('.')[0].split('_')[4]
    sm = pd.read_csv(f'timeseries/sensoryMotot_{sub}.csv') # reading somatosensory cortex as well
    if ts.shape[0] > 545:
        # z-score
        t = ts[:548]
        s = sm[:548] # sm cortex
        # add s to t
        tz = pd.concat([t,s['sensoryMotor']], axis=1)
        # append
        ts_total55.append(tz)
        sub55.append(sub)

    else:
        ts_total51.append(ts[2:])
        sub51.append(sub)

# remove items and subejcts
for i in range(len(ts_total55)):
    if ts_total55[i].shape[0] < 545:
        #print(i)
        #print(ts_total55[i].shape)
        sub55.pop(i)
        ts_total55.pop(i)
        
print(np.array(ts_total55).shape)        

(89, 548, 5)


In [53]:
stimList = pd.read_csv('behavior/StimList.csv')
np.array(stimList.Time)/2
dfGroups = pd.read_csv('behavior/SubGroupLists.csv', dtype={'Sub': 'string'})
dfGroups = dfGroups[['Sub','group', 'Eprime']]
for i in dfGroups.iterrows():
    dfGroups.at[i[0], 'Sub'] = "sub-" + str(i[1].Sub)

# lets take just the subjects in the B list and see something how the timeline looks in different conditions
dfA = dfGroups[(dfGroups.Eprime=="A")]
dfB = dfGroups[(dfGroups.Eprime=="B")]
# find the subjects that matched both A list and 55 list
bothList = set(dfA.Sub).intersection(sub55)
len(bothList) # 49 subjects
# extract their time series for amygdala and hippocampus
a_array = []
sub_a = []
for i, sub in enumerate(sub55):
    if sub in bothList: # check if subject is in the list
       # print(sub)
        a_array.append(ts_total55[i])
        sub_a.append(sub)
    else:
        print(f'sub {sub} was not found in bothList')

sub sub-011 was not found in bothList
sub sub-020 was not found in bothList
sub sub-022 was not found in bothList
sub sub-024 was not found in bothList
sub sub-025 was not found in bothList
sub sub-080 was not found in bothList
sub sub-089 was not found in bothList
sub sub-094 was not found in bothList
sub sub-100 was not found in bothList
sub sub-103 was not found in bothList
sub sub-1205 was not found in bothList
sub sub-1232 was not found in bothList
sub sub-126 was not found in bothList
sub sub-130 was not found in bothList
sub sub-135 was not found in bothList
sub sub-137 was not found in bothList
sub sub-140 was not found in bothList
sub sub-143 was not found in bothList
sub sub-144 was not found in bothList
sub sub-147 was not found in bothList
sub sub-148 was not found in bothList
sub sub-152 was not found in bothList
sub sub-154 was not found in bothList
sub sub-158 was not found in bothList
sub sub-161 was not found in bothList
sub sub-165 was not found in bothList
sub sub-16

In [54]:
bothListB = set(dfB.Sub).intersection(sub55)
len(bothListB) # 49 subjects
# extract their time series for amygdala and hippocampus
a_arrayB = []
sub_a_B = []
for i, sub in enumerate(sub55):
    if sub in bothListB: # check if subject is in the list
       # print(sub)
        a_arrayB.append(ts_total55[i])
        sub_a_B.append(sub)
    else:
        print(f'sub {sub} was not found in bothList')

sub sub-005 was not found in bothList
sub sub-008 was not found in bothList
sub sub-010 was not found in bothList
sub sub-011 was not found in bothList
sub sub-013 was not found in bothList
sub sub-016 was not found in bothList
sub sub-020 was not found in bothList
sub sub-022 was not found in bothList
sub sub-024 was not found in bothList
sub sub-025 was not found in bothList
sub sub-026 was not found in bothList
sub sub-027 was not found in bothList
sub sub-030 was not found in bothList
sub sub-032 was not found in bothList
sub sub-038 was not found in bothList
sub sub-043 was not found in bothList
sub sub-047 was not found in bothList
sub sub-048 was not found in bothList
sub sub-053 was not found in bothList
sub sub-055 was not found in bothList
sub sub-056 was not found in bothList
sub sub-059 was not found in bothList
sub sub-062 was not found in bothList
sub sub-063 was not found in bothList
sub sub-065 was not found in bothList
sub sub-066 was not found in bothList
sub sub-071 

In [55]:
a_array

[         sub           amg          hipp         vmpfc  sensoryMotor
 0    sub-005  1.308561e-10 -1.302570e-10  1.138559e-10 -1.036717e-10
 1    sub-005 -1.623076e+00 -2.385372e+00  1.995287e+00  8.305719e-01
 2    sub-005 -1.277589e+00 -1.250215e+00 -2.777410e-01  7.510483e-02
 3    sub-005  3.253449e-01  4.804404e-02 -1.227784e-01  1.323752e-01
 4    sub-005  1.529032e-01 -3.653774e-01 -7.280777e-01  4.734636e-01
 ..       ...           ...           ...           ...           ...
 543  sub-005 -3.815411e-01  9.469281e-02  1.162336e+00 -2.619127e-01
 544  sub-005  4.523365e-01 -3.226661e-02  1.235911e+00  3.564000e-01
 545  sub-005 -8.681527e-01 -5.228285e-01  1.073124e+00 -2.290380e-02
 546  sub-005 -8.621675e-02  4.744813e-01  9.118443e-01 -7.413827e-01
 547  sub-005  7.570587e-02  1.550082e-01  2.851057e-01  1.421980e-01
 
 [548 rows x 5 columns],
          sub           amg          hipp         vmpfc  sensoryMotor
 0    sub-008  4.829092e-10  1.619943e-10 -1.483641e-11 -1.0518

In [56]:
length = 8
hrf = 1 # adding diverge for to account for hrf
roi1 = 1#20
roi2 = 2#94
roi3 = 3#vmpfc
roi4 = 4 # SM cortex
subjects = []
trialNo = []
condition = []
coupling = []
coupAmgVmpfc = []
coupAmgSM = []
amg = [] # capture average amygdala activation
hipp = [] # capture hippocampus
vmpfc = [] # capture vmpfc
a_all = [] # grab all timeseries
b_all = []
for sub,v in enumerate(a_array):
        #print(sub)
        #print(v.shape)
        dist = []
        # plus US
        for i in stimList.iterrows():
            onset = int(i[1].Time /2) + hrf
            #print(onset)
            con = i[1].A
            #print(con)
            subj = sub_a[sub]
            #print(subj)
            trial = i[1].Trial

            #i = int(i)
            a = np.array(v)[onset:onset+length,roi1] 
            b = np.array(v)[onset:onset+length,roi2]
            c = np.array(v)[onset:onset+length,roi3]
            d = np.array(v)[onset:onset+length,roi4]
            coupling.append(scipy.stats.spearmanr(a,b)[0])
            coupAmgVmpfc.append(scipy.stats.spearmanr(a,c)[0])
            coupAmgSM.append(scipy.stats.spearmanr(a,d)[0])
            subjects.append(subj)
            condition.append(con)
            trialNo.append(trial)
            a_all.append(a)
            b_all.append(b)
            amg.append(np.mean(a))

In [58]:
dfCoupl = pd.DataFrame({'subject': subjects, 'trialNo': trialNo,
'condition': condition, 'coupling': coupling, 'amg': amg, 'amg_vmpfc': coupAmgVmpfc,
                       'amg_sm': coupAmgSM})

dfCoupl_first = dfCoupl#[dfCoupl.trialNo<=30] # take just first half
dfCoupl_first

,subject,trialNo,condition,coupling,amg,amg_vmpfc,amg_sm
0,sub-005,1,CSplusUS1,0.833333,-0.298303,-0.190476,0.000000
1,sub-005,2,CSminus1,0.809524,0.076128,0.476190,-0.023810
2,sub-005,3,CSplus1,0.642857,0.020805,0.071429,-0.023810
3,sub-005,4,CSplusUS1,0.809524,0.190180,0.666667,-0.500000
4,sub-005,5,CSminus1,0.976190,-0.080312,0.833333,-0.666667
...,...,...,...,...,...,...,...
3376,sub-205,65,CSminus2,0.309524,-0.212527,0.642857,0.023810
3377,sub-205,66,CSplus2,0.571429,0.223891,0.071429,0.857143
3378,sub-205,67,CSminus2,0.071429,0.020110,-0.166667,0.428571
3379,sub-205,68,CSplus2,0.285714,0.216831,0.428571,0.380952


In [59]:
# add second stimuli order (B)
subjects = []
trialNo = []
condition = []
coupling = []
amg = []
coupAmgVmpfc = []
coupAmgSM = []
for sub,v in enumerate(a_arrayB):
        #print(sub)
        #print(v.shape)
        dist = []
        # plus US
        for i in stimList.iterrows():
            onset = int(i[1].Time /2) + hrf
            #print(onset)
            con = i[1].B
            #print(con)
            subj = sub_a_B[sub]
            #print(subj)
            trial = i[1].Trial

            #i = int(i)
            a = np.array(v)[onset:onset+length,roi1] 
            b = np.array(v)[onset:onset+length,roi2] 
            c = np.array(v)[onset:onset+length,roi3]
            d = np.array(v)[onset:onset+length,roi4]
            coupling.append(scipy.stats.spearmanr(a,b)[0])
            coupAmgVmpfc.append(scipy.stats.spearmanr(a,c)[0])
            coupAmgSM.append(scipy.stats.spearmanr(a,d)[0])
            subjects.append(subj)
            condition.append(con)
            trialNo.append(trial)
            amg.append(np.mean(a))
        #distUS_b.append(dist)

In [60]:
dfCouplB = pd.DataFrame({'subject': subjects, 'trialNo': trialNo,
'condition': condition, 'coupling': coupling, 'amg':amg, 'amg_vmpfc': coupAmgVmpfc,
                       'amg_sm': coupAmgSM })

dfCouplB_first = dfCouplB#[dfCouplB.trialNo<=30] # take just first half
#dfCouplB_first

# combine both
dfBoth = pd.concat([dfCoupl_first, dfCouplB_first]).reset_index()
#dfBoth.to_csv('amg_hipp_fc_allTrials.csv', index=False)

In [67]:
dfBoth

,index,subject,trialNo,condition,coupling,amg,amg_vmpfc,amg_sm
0,0,sub-005,1,CSplusUS1,0.833333,-0.298303,-0.190476,0.000000
1,1,sub-005,2,CSminus1,0.809524,0.076128,0.476190,-0.023810
2,2,sub-005,3,CSplus1,0.642857,0.020805,0.071429,-0.023810
3,3,sub-005,4,CSplusUS1,0.809524,0.190180,0.666667,-0.500000
4,4,sub-005,5,CSminus1,0.976190,-0.080312,0.833333,-0.666667
...,...,...,...,...,...,...,...,...
5101,1720,sub-204,65,CSplus2,0.571429,0.038395,0.404762,-0.428571
5102,1721,sub-204,66,CSminus2,0.404762,0.361183,-0.023810,0.333333
5103,1722,sub-204,67,CSminus2,0.452381,0.003163,0.404762,0.238095
5104,1723,sub-204,68,CSplus2,0.833333,0.003435,0.857143,0.928571


In [66]:
dfBoth.to_csv('amg_hipp_fc_WholeROIs_SM.csv', index=False)

In [68]:
# now do calculations
scr_df = pd.read_csv('scr_clean.csv')
brain_df = pd.read_csv('amg_hipp_fc_WholeROIs_SM.csv')
df = pd.merge(scr_clean, brain_df, right_on=['subject','trialNo'], left_on=['sub','Event.Nr'])


In [69]:
df.head()

,sub,Condition,Event.Nr,CDA.AmpSum,pe,scr,index,subject,trialNo,condition,coupling,amg,amg_vmpfc,amg_sm
0,sub-189,CSplusUS1,1,0.2852,0.500000,0.2852,3036,sub-189,1,CSplusUS1,0.904762,0.476625,0.833333,-0.333333
1,sub-189,CSminus1,2,0.1033,-0.500000,0.1033,3037,sub-189,2,CSminus1,-0.380952,0.081692,0.428571,0.119048
2,sub-189,CSplus1,3,0.0783,-0.501659,0.0783,3038,sub-189,3,CSplus1,0.571429,-0.219659,0.690476,-0.047619
3,sub-189,CSplusUS1,4,0.1772,0.500009,0.1772,3039,sub-189,4,CSplusUS1,0.619048,0.006618,0.880952,0.142857
4,sub-189,CSminus1,5,0.0000,-0.498341,0.0000,3040,sub-189,5,CSminus1,0.833333,-0.188212,0.595238,0.642857


In [70]:
# Run Pymc model
# Encode 'sub' as integer indices
df['sub_idx'] = pd.Categorical(df['sub']).codes
n_subs = df['sub_idx'].nunique()

# Extract variables
pe = df['pe'].values
coupling = df['amg_sm'].values
amg = df['amg'].values
trialNo = df['trialNo'].values
sub_idx = df['sub_idx'].values

with pm.Model() as model_sm:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace_sm = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]
/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Output()

/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


In [72]:
az.summary(trace_sm, hdi_prob=.89)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_coupling,0.017,0.015,-0.007,0.040,0.000,0.000,8100.0,2799.0,1.0
beta_amg,0.009,0.022,-0.026,0.044,0.000,0.000,7838.0,2521.0,1.0
beta_trialNo,0.002,0.000,0.001,0.002,0.000,0.000,5280.0,2518.0,1.0
mu_a,-0.168,0.012,-0.188,-0.148,0.000,0.000,5448.0,2777.0,1.0
z_a[0],0.014,0.998,-1.561,1.626,0.012,0.019,7109.0,2981.0,1.0
...,...,...,...,...,...,...,...,...,...
a[60],-0.172,0.017,-0.198,-0.146,0.000,0.000,5126.0,2903.0,1.0
a[61],-0.167,0.016,-0.193,-0.142,0.000,0.000,4897.0,3079.0,1.0
a[62],-0.168,0.016,-0.193,-0.142,0.000,0.000,5270.0,3351.0,1.0
a[63],-0.168,0.017,-0.195,-0.143,0.000,0.000,4998.0,3140.0,1.0


## Framewise displacement

In [76]:
df_fd = pd.read_csv('fdData.csv')
df_fd.head()

,subject,cond,fd
0,sub-189,US,0.161557
1,sub-189,CS,0.111459
2,sub-189,CSm,0.135004
3,sub-205,US,0.107154
4,sub-205,CS,0.103271


In [91]:
# Create 'shock' column: 'yes' if cond == 'US', else 'no'
df_fd['shock'] = df_fd['cond'].apply(lambda x: 'yes' if x == 'US' else 'no')

# Group by subject and shock, then compute mean fd
df_fd_grouped = df_fd.groupby(['subject', 'shock'])['fd'].mean().reset_index()

df_fd_grouped

/tmp/ipykernel_3534793/1933991127.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_fd_grouped = df_fd.groupby(['subject', 'shock'])['fd'].mean().reset_index()


,subject,shock,fd
0,sub-010,no,0.167571
1,sub-010,yes,0.207947
2,sub-016,no,0.119993
3,sub-016,yes,0.131858
4,sub-026,no,0.289819
...,...,...,...
125,sub-203,yes,0.123519
126,sub-204,no,0.208439
127,sub-204,yes,0.216597
128,sub-205,no,0.097159


In [92]:
import bambi as bmb
model = bmb.Model('fd ~ shock + (1|subject)', df_fd_grouped)
results = model.fit()

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, shock, 1|subject_sigma, 1|subject_offset]
/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Output()

/home/oad4/.conda/envs/pymc_env/lib/python3.13/multiprocessing/popen_fork.py:67: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
sigma,0.028,0.003,0.023,0.033,0.000,0.0,1955.0,2723.0,1.00
Intercept,0.173,0.013,0.148,0.198,0.001,0.0,249.0,499.0,1.02
shock[yes],0.006,0.005,-0.004,0.015,0.000,0.0,8122.0,2772.0,1.00
1|subject_sigma,0.102,0.010,0.085,0.119,0.000,0.0,467.0,842.0,1.01
1|subject[sub-010],0.012,0.023,-0.034,0.053,0.001,0.0,768.0,1774.0,1.01
...,...,...,...,...,...,...,...,...,...
1|subject[sub-200],-0.024,0.023,-0.065,0.022,0.001,0.0,790.0,1761.0,1.01
1|subject[sub-202],-0.059,0.023,-0.104,-0.017,0.001,0.0,838.0,2104.0,1.00
1|subject[sub-203],-0.055,0.023,-0.100,-0.012,0.001,0.0,705.0,1557.0,1.01
1|subject[sub-204],0.035,0.022,-0.008,0.076,0.001,0.0,723.0,1476.0,1.01


# Adding diagnosis group to the original model

In [66]:
df = pd.read_csv('scr_amg_hipp_all.csv')
scr = pd.read_csv('behavior/SCR3.csv')
scr.head()

,Event.Nr,CDA.nSCR,CDA.Latency,CDA.AmpSum,CDA.SCR,CDA.ISCR,CDA.PhasicMax,CDA.Tonic,TTP.nSCR,TTP.Latency,TTP.AmpSum,Global.Mean,Global.MaxDeflection,Event.NID,Event.Name,Condition,group,sub
0,1,5,0.8435,0.2852,0.0003,0.1339,8.1296,2.3324,1,3.9335,0.5884,2.4822,0.5884,5,5,CSplusUS1,HC,189
1,2,4,0.7335,0.1033,0.0012,0.4737,0.3046,4.0029,0,NaN,0.0000,4.3933,0.0000,5,5,CSminus1,HC,189
2,3,3,2.9835,0.0783,0.0008,0.3237,0.1352,3.9579,1,2.8335,0.0260,3.9900,0.0154,5,5,CSplus1,HC,189
3,4,1,3.4935,0.1772,0.0002,0.0993,7.0748,3.8756,1,3.5335,0.5186,3.9212,0.5186,5,5,CSplusUS1,HC,189
4,5,0,NaN,0.0000,0.0004,0.1532,0.1604,4.2513,0,NaN,0.0000,4.3461,0.0000,5,5,CSminus1,HC,189


In [67]:
scr = scr[['sub','group', 'Event.Nr']]
scr['sub'] = scr['sub'].astype('string')
for i in scr.iterrows():
    if len(i[1]['sub'])<=2:
        #print(i[1]['sub'])
        sub = 'sub-0' + str(i[1]['sub'])
    else:
        sub = 'sub-' + str(i[1]['sub'])
    #print(sub)
    scr.at[i[0], 'sub'] = sub
    

In [68]:
df = pd.merge(df, scr, left_on=['sub', 'Event.Nr'], right_on=['sub', 'Event.Nr'])
df

,sub,Condition,Event.Nr,CDA.AmpSum,expected_value,pe,scr,index,subject,trialNo,condition,coupling,amg,amg_vmpfc,group
0,sub-189,CSplusUS1,1,0.2852,0.773082,0.500000,0.2852,3036,sub-189,1,CSplusUS1,0.904762,0.476625,0.833333,HC
1,sub-189,CSminus1,2,0.1033,0.770982,-0.500000,0.1033,3037,sub-189,2,CSminus1,-0.380952,0.081692,0.428571,HC
2,sub-189,CSplus1,3,0.0783,0.772029,-0.500672,0.0783,3038,sub-189,3,CSplus1,0.571429,-0.219659,0.690476,HC
3,sub-189,CSplusUS1,4,0.1772,0.773079,0.500002,0.1772,3039,sub-189,4,CSplusUS1,0.619048,0.006618,0.880952,HC
4,sub-189,CSminus1,5,0.0000,0.769936,-0.499328,0.0000,3040,sub-189,5,CSminus1,0.833333,-0.188212,0.595238,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,sub-152,CSplus2,65,0.0000,0.003328,-0.004241,0.0000,547,sub-152,65,CSplus2,0.428571,0.160432,0.880952,HC
4481,sub-152,CSminus2,66,0.0000,0.277273,-0.421484,0.0000,548,sub-152,66,CSminus2,0.357143,0.066549,0.309524,HC
4482,sub-152,CSminus2,67,0.0000,0.189445,-0.277331,0.0000,549,sub-152,67,CSminus2,0.619048,0.262597,-0.047619,HC
4483,sub-152,CSplus2,68,0.0000,0.002758,-0.003514,0.0000,550,sub-152,68,CSplus2,0.761905,-0.240072,0.071429,HC


In [70]:
az.summary(trace_group, hdi_prob=.89, 
           var_names = ['beta_VPTSD','beta_coupling','beta_amg'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_VPTSD,0.007,0.497,-0.828,0.774,0.006,0.009,7344.0,2665.0,1.01
beta_coupling,0.090,0.020,0.057,0.121,0.000,0.000,4327.0,2849.0,1.00
beta_amg,0.020,0.024,-0.018,0.057,0.000,0.000,6172.0,2493.0,1.00


In [69]:
# Encode 'sub' as integer indices
df['sub_idx'] = pd.Categorical(df['sub']).codes
n_subs = df['sub_idx'].nunique()
df['group'] = pd.Categorical(df['group']).codes
# Extract variables
pe = df['pe'].values
coupling = df['coupling'].values
amg = df['amg'].values
trialNo = df['trialNo'].values
sub_idx = df['sub_idx'].values
# combine contols
df['is_VPTSD'] = (df['group'] == 'VPTSD').astype(int)


with pm.Model() as model_group:
    # Priors for fixed effects
    beta_coupling = pm.Normal('beta_coupling', 0, 1)
    beta_amg = pm.Normal('beta_amg', 0, 1)
    beta_trialNo = pm.Normal('beta_trialNo', 0, 1)
    
    beta_VPTSD = pm.Normal('beta_VPTSD', 0, .5)
    
    # Random intercepts
    mu_a = pm.Normal('mu_a', 0, 1)
    sigma_a = pm.HalfNormal('sigma_a', 1)
    z_a = pm.Normal('z_a', 0, 1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Linear predictor
    mu = (
        a[sub_idx] +
        beta_coupling * df['coupling'].values +
        beta_amg * df['amg'].values +
        beta_trialNo * df['trialNo'].values +
        beta_VPTSD * df['is_VPTSD'].values
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', 1)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=df['pe'].values)

    # Sampling
    trace_group = pm.sample(chains=4,
                            return_inferencedata=True,
                           idata_kwargs={"log_likelihood": True})

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, beta_VPTSD, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.


## All three groups

In [71]:
df['group'] = pd.Categorical(df['group'], categories=['HC', 'VPTSD', 'VCC'])

# Create dummy variables with HC as the reference group
df['group_VPTSD'] = (df['group'] == 'VPTSD').astype(int)
df['group_VCC'] = (df['group'] == 'VCC').astype(int)
group_VPTSD = df['group_VPTSD'].values
group_VCC = df['group_VCC'].values

with pm.Model() as model_3groups:
    # Fixed effect priors
    beta_coupling = pm.Normal('beta_coupling', 0, 1)
    beta_amg = pm.Normal('beta_amg', 0, 1)
    beta_trialNo = pm.Normal('beta_trialNo', 0, 1)
    
    beta_VPTSD = pm.Normal('beta_VPTSD', 0, 0.5)
    beta_VCC = pm.Normal('beta_VCC', 0, 0.5)

    # Random intercepts
    mu_a = pm.Normal('mu_a', 0, 1)
    sigma_a = pm.HalfNormal('sigma_a', 1)
    z_a = pm.Normal('z_a', 0, 1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Linear predictor
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo +
        beta_VPTSD * group_VPTSD +
        beta_VCC * group_VCC
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', 1)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=pe)

    # Sampling
    trace_3groups = pm.sample(chains=4,
                              return_inferencedata=True,
                              idata_kwargs={"log_likelihood": True})


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, beta_VPTSD, beta_VCC, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.


In [73]:
az.summary(trace_3groups, hdi_prob=.89, 
           var_names = ['beta_VPTSD','beta_VCC','beta_coupling','beta_amg'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_VPTSD,-0.005,0.503,-0.845,0.754,0.006,0.010,7344.0,2493.0,1.0
beta_VCC,-0.013,0.505,-0.788,0.839,0.006,0.009,6648.0,2876.0,1.0
beta_coupling,0.091,0.020,0.058,0.122,0.000,0.000,5011.0,2881.0,1.0
beta_amg,0.019,0.023,-0.018,0.055,0.000,0.000,6850.0,2607.0,1.0


### Amygala vmPFC coupling


In [74]:
df['group'] = pd.Categorical(df['group'], categories=['HC', 'VPTSD', 'VCC'])

# Create dummy variables with HC as the reference group
df['group_VPTSD'] = (df['group'] == 'VPTSD').astype(int)
df['group_VCC'] = (df['group'] == 'VCC').astype(int)
group_VPTSD = df['group_VPTSD'].values
group_VCC = df['group_VCC'].values

with pm.Model() as model_3groups_vmpfc:
    # Fixed effect priors
    beta_coupling = pm.Normal('beta_coupling', 0, 1)
    beta_amg = pm.Normal('beta_amg', 0, 1)
    beta_trialNo = pm.Normal('beta_trialNo', 0, 1)
    
    beta_VPTSD = pm.Normal('beta_VPTSD', 0, 0.5)
    beta_VCC = pm.Normal('beta_VCC', 0, 0.5)

    # Random intercepts
    mu_a = pm.Normal('mu_a', 0, 1)
    sigma_a = pm.HalfNormal('sigma_a', 1)
    z_a = pm.Normal('z_a', 0, 1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Linear predictor
    mu = (
        a[sub_idx] +
        beta_coupling * df['amg_vmpfc'] +
        beta_amg * amg +
        beta_trialNo * trialNo +
        beta_VPTSD * group_VPTSD +
        beta_VCC * group_VCC
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', 1)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=pe)

    # Sampling
    trace_3groups_vmpfc = pm.sample(chains=4,
                              return_inferencedata=True,
                              idata_kwargs={"log_likelihood": True})


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, beta_VPTSD, beta_VCC, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.


In [75]:
az.summary(trace_3groups_vmpfc, hdi_prob=.89,
           var_names = ['beta_VPTSD','beta_VCC','beta_coupling','beta_amg'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_VPTSD,-0.009,0.490,-0.793,0.776,0.006,0.009,6555.0,2640.0,1.0
beta_VCC,-0.008,0.497,-0.865,0.716,0.006,0.009,6721.0,2960.0,1.0
beta_coupling,-0.045,0.016,-0.070,-0.018,0.000,0.000,6226.0,3185.0,1.0
beta_amg,0.015,0.023,-0.019,0.053,0.000,0.001,6345.0,2575.0,1.0


# Testing differen HRF delays
- TR=0 (0 seconds delay)
- TR=2 (4 seconds delay)

We will generate data on coupling of amygdala hippocampus and amygdala vmpfc on each potential delay and run the relevant statistical models

In [10]:
# grabbing the relevant arrays
a_array = np.load('a_array.npy', allow_pickle=True)
sub_a = np.load('sub_a.npy', allow_pickle=True)
a_arrayB = np.load('a_arrayB.npy', allow_pickle=True)
sub_a_B = np.load('sub_a_B.npy', allow_pickle=True)

# load stimlist
stimList = pd.read_csv('behavior/StimList.csv')

In [29]:
import numpy as np
import pandas as pd
import scipy.stats

def extract_coupling_data(a_array, sub_a, stimList, hrf=1, length=8, roi1=1, roi2=2, roi3=3, source='A'):
    subjects = []
    trialNo = []
    condition = []
    coupling = []
    coupAmgVmpfc = []
    amg = []
    a_all = []
    b_all = []

    for sub, v in enumerate(a_array):
        for _, row in stimList.iterrows():
            onset = int(row.Time / 2) + hrf
            con = row.A
            subj = sub_a[sub]
            trial = row.Trial

            try:
                a = np.array(v)[onset:onset+length, roi1]
                b = np.array(v)[onset:onset+length, roi2]
                c = np.array(v)[onset:onset+length, roi3]

                coupling.append(scipy.stats.spearmanr(a, b)[0])
                coupAmgVmpfc.append(scipy.stats.spearmanr(a, c)[0])
                subjects.append(subj)
                condition.append(con)
                trialNo.append(trial)
                a_all.append(a)
                b_all.append(b)
                amg.append(np.mean(a))
            except IndexError:
                # Optional: skip if onset + length exceeds timeseries length
                continue

    df = pd.DataFrame({
        'sub': subjects,
        'trialNo': trialNo,
        'condition': condition,
        'coupling': coupling,
        'amg': amg,
        'amg_vmpfc': coupAmgVmpfc,
        'source': source,
        'hrf': hrf
    })

    return df

In [30]:
# For HRF = 0
df_A_hrf0 = extract_coupling_data(a_array, sub_a, stimList, hrf=0, source='A')
df_B_hrf0 = extract_coupling_data(a_arrayB, sub_a_B, stimList, hrf=0, source='B')
df_combined = pd.concat([df_A_hrf0, df_B_hrf0], ignore_index=True)

In [31]:
# load original data (has pe and scr)
df_org = pd.read_csv('scr_amg_hipp_all.csv')[['sub','trialNo','pe']]
df_org

,sub,trialNo,pe
0,sub-189,1,0.500000
1,sub-189,2,-0.500000
2,sub-189,3,-0.500672
3,sub-189,4,0.500002
4,sub-189,5,-0.499328
...,...,...,...
4480,sub-152,65,-0.004241
4481,sub-152,66,-0.421484
4482,sub-152,67,-0.277331
4483,sub-152,68,-0.003514


In [32]:
df = pd.merge(df_combined, df_org)
df

,sub,trialNo,condition,coupling,amg,amg_vmpfc,source,hrf,pe
0,sub-010,1,CSplusUS1,0.857143,0.037732,0.690476,A,0,0.500000
1,sub-010,2,CSminus1,0.714286,0.197202,0.571429,A,0,-0.500000
2,sub-010,3,CSplus1,0.642857,0.095099,0.333333,A,0,-0.626709
3,sub-010,4,CSplusUS1,0.309524,0.141916,0.333333,A,0,0.553584
4,sub-010,5,CSminus1,0.857143,0.045487,0.523810,A,0,-0.373291
...,...,...,...,...,...,...,...,...,...
4480,sub-204,65,CSminus2,0.666667,-0.009111,0.428571,B,0,-0.003264
4481,sub-204,66,CSplus2,0.666667,0.290669,0.333333,B,0,-0.430361
4482,sub-204,67,CSminus2,0.309524,0.102712,0.119048,B,0,-0.277339
4483,sub-204,68,CSplus2,0.809524,0.039305,0.928571,B,0,-0.002706


In [33]:
# run analysis
df['sub_idx'] = pd.Categorical(df['sub']).codes
n_subs = df['sub_idx'].nunique()

# Extract variables
pe = df['pe'].values
coupling = df['coupling'].values
amg = df['amg'].values
trialNo = df['trialNo'].values
sub_idx = df['sub_idx'].values

with pm.Model() as model:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.


In [34]:
az.summary(trace, hdi_prob=.89,
           var_names = ['beta_amg','beta_coupling', 'beta_trialNo'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_amg,0.056,0.023,0.020,0.094,0.0,0.0,5701.0,2642.0,1.0
beta_coupling,0.099,0.020,0.069,0.131,0.0,0.0,3703.0,2864.0,1.0
beta_trialNo,0.001,0.000,0.001,0.002,0.0,0.0,4289.0,2576.0,1.0


In [35]:
# now for vmPFC
coupling = df['amg_vmpfc'].values
with pm.Model() as model_v:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace_v = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


In [36]:
az.summary(trace_v, hdi_prob=.89,
           var_names = ['beta_amg','beta_coupling', 'beta_trialNo'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_amg,0.055,0.024,0.015,0.092,0.0,0.0,5476.0,2853.0,1.0
beta_coupling,-0.049,0.016,-0.073,-0.024,0.0,0.0,4213.0,2360.0,1.0
beta_trialNo,0.001,0.000,0.001,0.002,0.0,0.0,3412.0,2824.0,1.0


In [30]:
# For HRF = 0
df_A_hrf0 = extract_coupling_data(a_array, sub_a, stimList, hrf=0, source='A')
df_B_hrf0 = extract_coupling_data(a_arrayB, sub_a_B, stimList, hrf=0, source='B')
df_combined = pd.concat([df_A_hrf0, df_B_hrf0], ignore_index=True)

In [31]:
# load original data (has pe and scr)
df_org = pd.read_csv('scr_amg_hipp_all.csv')[['sub','trialNo','pe']]
df_org

,sub,trialNo,pe
0,sub-189,1,0.500000
1,sub-189,2,-0.500000
2,sub-189,3,-0.500672
3,sub-189,4,0.500002
4,sub-189,5,-0.499328
...,...,...,...
4480,sub-152,65,-0.004241
4481,sub-152,66,-0.421484
4482,sub-152,67,-0.277331
4483,sub-152,68,-0.003514


In [32]:
df = pd.merge(df_combined, df_org)
df

,sub,trialNo,condition,coupling,amg,amg_vmpfc,source,hrf,pe
0,sub-010,1,CSplusUS1,0.857143,0.037732,0.690476,A,0,0.500000
1,sub-010,2,CSminus1,0.714286,0.197202,0.571429,A,0,-0.500000
2,sub-010,3,CSplus1,0.642857,0.095099,0.333333,A,0,-0.626709
3,sub-010,4,CSplusUS1,0.309524,0.141916,0.333333,A,0,0.553584
4,sub-010,5,CSminus1,0.857143,0.045487,0.523810,A,0,-0.373291
...,...,...,...,...,...,...,...,...,...
4480,sub-204,65,CSminus2,0.666667,-0.009111,0.428571,B,0,-0.003264
4481,sub-204,66,CSplus2,0.666667,0.290669,0.333333,B,0,-0.430361
4482,sub-204,67,CSminus2,0.309524,0.102712,0.119048,B,0,-0.277339
4483,sub-204,68,CSplus2,0.809524,0.039305,0.928571,B,0,-0.002706


In [33]:
# run analysis
df['sub_idx'] = pd.Categorical(df['sub']).codes
n_subs = df['sub_idx'].nunique()

# Extract variables
pe = df['pe'].values
coupling = df['coupling'].values
amg = df['amg'].values
trialNo = df['trialNo'].values
sub_idx = df['sub_idx'].values

with pm.Model() as model:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.


In [34]:
az.summary(trace, hdi_prob=.89,
           var_names = ['beta_amg','beta_coupling', 'beta_trialNo'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_amg,0.056,0.023,0.020,0.094,0.0,0.0,5701.0,2642.0,1.0
beta_coupling,0.099,0.020,0.069,0.131,0.0,0.0,3703.0,2864.0,1.0
beta_trialNo,0.001,0.000,0.001,0.002,0.0,0.0,4289.0,2576.0,1.0


In [35]:
# now for vmPFC
coupling = df['amg_vmpfc'].values
with pm.Model() as model_v:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace_v = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


In [36]:
az.summary(trace_v, hdi_prob=.89,
           var_names = ['beta_amg','beta_coupling', 'beta_trialNo'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_amg,0.055,0.024,0.015,0.092,0.0,0.0,5476.0,2853.0,1.0
beta_coupling,-0.049,0.016,-0.073,-0.024,0.0,0.0,4213.0,2360.0,1.0
beta_trialNo,0.001,0.000,0.001,0.002,0.0,0.0,3412.0,2824.0,1.0


## Now for HRF = 2 (4 seconds)

In [37]:
# For HRF = 2
df_A_hrf2 = extract_coupling_data(a_array, sub_a, stimList, hrf=2, source='A')
df_B_hrf2 = extract_coupling_data(a_arrayB, sub_a_B, stimList, hrf=2, source='B')
df_combined = pd.concat([df_A_hrf2, df_B_hrf2], ignore_index=True)

In [38]:
# load original data (has pe and scr)
df_org = pd.read_csv('scr_amg_hipp_all.csv')[['sub','trialNo','pe']]
df_org

,sub,trialNo,pe
0,sub-189,1,0.500000
1,sub-189,2,-0.500000
2,sub-189,3,-0.500672
3,sub-189,4,0.500002
4,sub-189,5,-0.499328
...,...,...,...
4480,sub-152,65,-0.004241
4481,sub-152,66,-0.421484
4482,sub-152,67,-0.277331
4483,sub-152,68,-0.003514


In [39]:
df = pd.merge(df_combined, df_org)
df

,sub,trialNo,condition,coupling,amg,amg_vmpfc,source,hrf,pe
0,sub-010,1,CSplusUS1,0.809524,-0.027128,0.785714,A,2,0.500000
1,sub-010,2,CSminus1,0.666667,0.183908,0.523810,A,2,-0.500000
2,sub-010,3,CSplus1,0.809524,0.216987,0.190476,A,2,-0.626709
3,sub-010,4,CSplusUS1,0.285714,-0.047552,0.642857,A,2,0.553584
4,sub-010,5,CSminus1,0.523810,0.038555,-0.214286,A,2,-0.373291
...,...,...,...,...,...,...,...,...,...
4480,sub-204,65,CSminus2,0.785714,0.080191,0.761905,B,2,-0.003264
4481,sub-204,66,CSplus2,0.285714,0.256768,0.285714,B,2,-0.430361
4482,sub-204,67,CSminus2,0.619048,-0.008220,0.500000,B,2,-0.277339
4483,sub-204,68,CSplus2,0.833333,0.020083,0.380952,B,2,-0.002706


In [40]:
# run analysis
df['sub_idx'] = pd.Categorical(df['sub']).codes
n_subs = df['sub_idx'].nunique()

# Extract variables
pe = df['pe'].values
coupling = df['coupling'].values
amg = df['amg'].values
trialNo = df['trialNo'].values
sub_idx = df['sub_idx'].values

with pm.Model() as model:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


In [41]:
az.summary(trace, hdi_prob=.89,
           var_names = ['beta_amg','beta_coupling', 'beta_trialNo'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_amg,0.076,0.023,0.039,0.114,0.0,0.0,5526.0,2697.0,1.0
beta_coupling,0.068,0.019,0.037,0.100,0.0,0.0,4507.0,3036.0,1.0
beta_trialNo,0.001,0.000,0.001,0.002,0.0,0.0,4449.0,3179.0,1.0


In [42]:
# now for vmPFC
coupling = df['amg_vmpfc'].values
with pm.Model() as model_v:
    
    # Fixed effects
    beta_coupling = pm.Normal('beta_coupling', mu=0, sigma=1)
    beta_amg = pm.Normal('beta_amg', mu=0, sigma=1)
    beta_trialNo = pm.Normal('beta_trialNo', mu=0, sigma=1)

    # Hyperpriors
    mu_a = pm.Normal('mu_a', mu=0, sigma=1)
    sigma_a = pm.HalfNormal('sigma_a', sigma=1)

    # Non-centered random intercepts
    z_a = pm.Normal('z_a', mu=0, sigma=1, shape=n_subs)
    a = pm.Deterministic('a', mu_a + z_a * sigma_a)

    # Expected value of outcome
    mu = (
        a[sub_idx] +
        beta_coupling * coupling +
        beta_amg * amg +
        beta_trialNo * trialNo
    )

    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = pm.Normal('pe', mu=mu, sigma=sigma, observed=pe)
    trace_v = pm.sample(chains=4, return_inferencedata=True,
                   idata_kwargs={"log_likelihood": True},
                  )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_coupling, beta_amg, beta_trialNo, mu_a, sigma_a, z_a, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


In [43]:
az.summary(trace_v, hdi_prob=.89,
           var_names = ['beta_amg','beta_coupling', 'beta_trialNo'])

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_amg,0.075,0.024,0.035,0.112,0.0,0.0,5383.0,2338.0,1.01
beta_coupling,-0.022,0.015,-0.047,0.003,0.0,0.0,6420.0,2901.0,1.00
beta_trialNo,0.001,0.000,0.001,0.002,0.0,0.0,4141.0,2986.0,1.00
